<a href="https://colab.research.google.com/github/oda251/mojiokoshi_tools/blob/main/Whisper_%E6%96%87%E5%AD%97%E8%B5%B7%E3%81%93%E3%81%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **使用方法**
#### 1. 上部メニューの「ランタイム」「ランタイムのタイプを変更」から、「T4 GPU」を選択してください
#### 2. 上から順番に、指示に従って入力等を行ってください
#### 3. 最後に、上部メニューの「ランタイム」から「すべてのセルを実行」をクリックすると、output.txtに結果が格納されます。

In [ ]:
#@title 必要なライブラリのインストール
!pip install -U openai-whisper
!sudo apt update && sudo apt install ffmpeg
!pip install pydub

import subprocess
import whisper
from pydub import AudioSegment
import os
from google.colab import files

In [ ]:
#@title Whisperモデルを選択
#@markdown 1. モデルを選択してください。大きいほど精度と処理時間が大きくなります
モデル = "large" #@param ["large", "medium", "small", "base", "tiny"]
model = whisper.load_model(モデル)

In [ ]:
#@title ファイルのアップロード
#@markdown - 音声ファイルをアップロードしてください
uploaded = files.upload()
audio_path = list(uploaded)[0]

if os.path.isfile(audio_path):
  print("アップロードが完了しました")
else:
  print("入力されたパスにファイルが見つかりませんでした")

In [ ]:
#@title 実行
#@markdown 1. 上部メニューの「ランタイム」から「すべてのセルを実行」をクリック
if not os.path.isfile(audio_path):
  print("ファイルが正しくアップロードされていません")

def convert_m4a_to_mp3(file_path, output_path):
    # ffmpegを使用してm4aをmp3に変換
    subprocess.run(['ffmpeg', '-i', file_path, output_path])

def transcribe_audio(file_path):
    # ここでは、modelは既にロードされていると想定
    global model
    sound = AudioSegment.from_file(file_path)
    duration = sound.duration_seconds
    result = []
    for i, j in enumerate(range(0, int(duration), 25)):
        left = j
        if left > 0:
          left += 3
        right = (j + 25)
        left *= 1000
        right *= 1000
        tgt = "tmp.mp3"
        sound[left:right].export(tgt, format="mp3")
        result.append(model.transcribe(tgt)["text"])
    return result

transcription_result = transcribe_audio(audio_path)

# 保存先の指定
output_path = '/content/'

# ディレクトリが存在しない場合は作成
if not os.path.exists(output_path):
    os.makedirs(output_path)

# 保存
export_file = output_path + "output.txt"
with open(export_file, 'w') as f:
    for txt in transcription_result:
      f.write(txt)
      f.write('\n')

print(f"ファイルが保存されました: {export_file}")